In [15]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.sparse import diags
from sympy import symbols as symb
from scipy import linalg

In [38]:
#LU factorization

def LUfunc(M):
    A = M.astype(float) #To make sure we have float division
    N = np.shape(A)[0]
    for k in range(N-1):
        if A[k,k] == 0:
            print("Breakdown due to: Zero pivot")
            break
        for i in range(k+1,N):
            A[i,k] = A[i,k]/A[k,k]
            for j in range(k+1,N):
                A[i,j] = A[i,j] - A[i,k]*A[k,j]
    return A

def LUfuncSparse(M):
    A = sparse.csr_matrix(M) #To make sure we have float division
    N = np.shape(M)[0]
    for k in range(N-1):
        if A[k,k] == 0:
            print("Breakdown due to: Zero pivot")
            break
        for i in range(k+1,N):
            A[i,k] = A[i,k]/A[k,k]
            for j in range(k+1,N):
                A[i,j] = A[i,j] - A[i,k]*A[k,j]
    return A

def LUfunc2(A):
    L = np.tril(A,-1)+np.eye(np.shape(A)[0])
    U = np.triu(A)
    return L , U

In [18]:
#Substitution

#Forward     Works for a LU decomposed matrix from LUfunc      L is NxN matrix, f is N vector
def Forsub(A,f):
    N = np.shape(A)[0]
    y = np.array([f[0]]) #first element is always just f[0] due to lower tridiagonal
    L = A.astype(float)
    for i in range(N):
        L[i,i] = 1   #Make sure diagonal is 1
    for i in range(1,N):
        y=np.append(y,f[i]-np.dot(L[i,:i],y[:i]))
    return y
    
#backward    Use y from Forsub
def Backsub(A,y):
    N = np.shape(A)[0]
    U = A.astype(float)
    u = np.zeros(N)
    u[-1] = y[-1]/U[-1,-1]   #manually add first element to u
    for i in reversed(range(N-1)):
        u[i] = ( y[i]-np.dot(U[i,i:N],u[i:N]) )/U[i,i]
    return u

In [39]:
#Final direct solver

def DirSolver(A,f):
    LU = LUfunc(A)
    y = Forsub(LU,f)
    u = Backsub(LU,y)
    return u

def DirSolverSparse(A,f):
    LU = LUfuncSparse(A)
    y = Forsub(LU,f)
    u = Backsub(LU,y)
    return u

In [24]:
#Example:
A = np.array([[3,4,2],[1,2,4],[8,7,6]])
f = np.array([59,51,137])
solution = np.array([5.,7.,8.])

u = DirSolver(A,f)
print(u)
print(u.round(3)==solution.round(3))

[5. 7. 8.]
[ True  True  True]


In [23]:
#Example2:
N = 300
C = np.random.rand(N,N)
#print(C)
solution = np.random.rand(N)
#print(solution)
h = np.matmul(C,solution)

u = DirSolver(C,h)
#print(u)
#print(solution)
#print(u.round(4)==solution.round(4))

In [40]:
#Test sparse3:
import time
n = [5,50,100,150,200]
t = []
ts = []  
avg = 3
for i in n:
    timeavg = []
    for j in range(avg):
        D = np.random.rand(i,i)
        j = np.random.rand(i)
        t1 = time.time()
        u = DirSolver(D,j)
        t2 = time.time()
        timeavg = np.append(timeavg,t2-t1)
    t = np.append(t,np.sum(timeavg)/avg)
    
for i in n:
    timeavg = []
    for j in range(avg):
        D = np.random.rand(i,i)
        j = np.random.rand(i)
        t1 = time.time()
        u = DirSolverSparse(D,j)
        t2 = time.time()
        timeavg = np.append(timeavg,t2-t1)
    ts = np.append(t,np.sum(timeavg)/avg)

NotImplementedError: subtracting a sparse matrix from a nonzero scalar is not supported

In [37]:
print(t)

[5.75145086e-04 3.58779430e-02 2.22449223e-01 7.26496538e-01
 1.75654769e+00]


In [171]:
#SSOR method

def SORit(A,u,w,f):
    N = np.size(u)
    s0 = u[0]
    u[0] = (f[0] - np.dot(A[0,1:],u[1:]))/A[0,0]
    u[0] = (1-w)*s0+w*u[0]
    for i in range(1,N-1):
        s = u[i]
        u[i] = (f[i]-np.dot(A[i,:i-1],u[:i-1])-np.dot(A[i,i+1:],u[i+1:]))/A[i,i]
        u[i] = (1-w)*s+w*u[i]
    return u

def SOR(A,u,w,f,N):
    u0 = u
    for i in range(N):
        print(u0)
        u0 = SORit(A,u0,w,f)
    return u0


In [158]:
#Example3:
N=4
d=10
D = np.random.rand(N,N)
j = np.random.rand(N)
u = DirSolver(D,j)

#SSOR
u0 = np.zeros(N)
us = SOR(D,u0,1,j,5)

[0. 0. 0. 0.]
[ 6.30979537  1.7813246  -6.98370489  0.        ]
[ 37.90841327   5.34352048 -46.93477624   0.        ]
[ 234.60282493   25.72146368 -295.62136437    0.        ]
[ 1463.29870143   152.56965565 -1849.0980635      0.        ]


In [159]:
print(u)
print(us)

[-1.05627614  2.7984555  -0.35482717  0.58506365]
[  9139.69871407    944.95541334 -11554.59865235      0.        ]


In [2]:
from scipy import sparse
A = [[1,0,1],[0,4,0],[0,0,5]]
As = sparse.csr_matrix(A) #make sparse

In [12]:
print(As[2,2])

5
